In [1]:
# !pip install geopy

In [2]:
from geopy.geocoders import Nominatim
import pandas as pd
import csv
from geopy.extra.rate_limiter import RateLimiter
import concurrent.futures

geolocator = Nominatim(user_agent="your_app_name", timeout=10)
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)

In [3]:

from geopy.exc import GeocoderQueryError

def get_coord(address):
    try:
        location = geolocator.geocode(address)
        if location:
            print(location.address)
            return location.address, location.latitude, location.longitude
        else:
            # Handle the case where the location is not found
            return address, None, None
    except GeocoderQueryError as e:
        # Handle the geocoding error
        print(f"Geocoding error: {e}")
        return address, None, None

In [4]:
import numpy as np
def get_detail_add():
    # read file csv
    data_file_path = "/home/nmh/ds/project/data_validation/cleaned_concatenated_data.csv"
#     data = pd.read_csv(data_file_path, skiprows=range(1, 1000))
    data = pd.read_csv(data_file_path)
    add_list = []

    for index, row in data.iterrows():
        city = row['city']
        district = row['district']
        ward = row['ward']
        street = row['street']

        if street is np.nan:
            add = f"{ward} {district} {city}"
        else:
            add = f"{street} {ward} {district} {city}"
        add_list.append(add)
    return add_list

In [5]:
# !rm -r /kaggle/working/output.csv

In [6]:
import time
def process_address(address):
    try:
        return get_coord(address)
    except Exception as e:
        print(f"Exception occurred while processing address {address}: {e}")
        return address, None, None

In [7]:
def main():
    add_list = get_detail_add()
    csv_file_path = '/home/nmh/ds/project/data_coordinating/data_with_coordination.csv'

    with open(csv_file_path, mode='a', newline='') as file:
        writer = csv.writer(file)
        with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
            # Submit tasks to the executor
            future_to_address = {executor.submit(process_address, address): address for address in add_list}

            for future in concurrent.futures.as_completed(future_to_address):
                address = future_to_address[future]
                try:
                    result = future.result()
                    writer.writerow(result)
                except Exception as e:
                    print(f"Exception occurred in future for address {address}: {e}")

In [8]:
# import time

# add_list = get_detail_add()

# csv_file_path = '/home/nmh/ds/project/data_coordinating/data_with_coordination.csv'
# i = 0

# with open(csv_file_path, mode='a', newline='') as file:
#     # Tạo đối tượng ghi CSV
#     writer = csv.writer(file)
#     for a in add_list:
#         print(i)
#         i += 1
#         add, la, long = get_coord(a)
#         print(add)
#         writer.writerow([add, la, long])
#         time.sleep(2)

In [9]:
if __name__ == "__main__":
    main()

Phố Dương Đình Nghệ, Phường Yên Hòa, Quận Cầu Giấy, Thành phố Hà Nội, 10298, Việt Nam
Chung cư Hà Nội Homeland Long Biên, Đường nội bộ, Phường Thượng Thanh, Quận Long Biên, Thành phố Hà Nội, 08443, Việt Nam
Đường Võ Chí Công, Phường Phú Thượng, Quận Tây Hồ, Thành phố Hà Nội, 11207, Việt Nam
Đường Quang Trung, Phường Quang Trung, Quận Hà Đông, Thành phố Hà Nội, 12011, Việt Nam
Đường Quang Trung, Phường Quang Trung, Quận Hà Đông, Thành phố Hà Nội, 12011, Việt Nam
Đường Lê Quang Đạo, Phường Phú Đô, Quận Nam Từ Liêm, Thành phố Hà Nội, 12011, Việt Nam
Phố Quan Nhân, Phường Nhân Chính, Quận Thanh Xuân, Thành phố Hà Nội, 11513, Việt Nam
Đường Trung Văn, Phường Trung Văn, Quận Nam Từ Liêm, Thành phố Hà Nội, 12011, Việt Nam
Đường Phùng Hưng, Phường Văn Quán, Quận Hà Đông, Thành phố Hà Nội, 12011, Việt Nam
Đường Ngọc Hồi, Tổ Pin, Thị trấn Văn Điển, Huyện Thanh Trì, Thành phố Hà Nội, 11718, Việt Nam
Phố Chu Văn An, Vạn Phúc, Phường Vạn Phúc, Quận Hà Đông, Thành phố Hà Nội, 12011, Việt Nam
Đường G